In [1]:
import numpy as np
from sklearn.datasets import load_boston, load_iris, load_diabetes, load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
# skoptを使ってみる
# skoptのuserwarningへの対応
warnings.filterwarnings('ignore')

TEST_SIZE = 0.3


def calculate_score(estimator, X_train, X_test, y_train, y_test):
    estimator.fit(X_train, y_train)
    return estimator.score(X_test, y_test)


def scale(X_train, X_test):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return (X_train, X_test)


def iris():
    X_train, X_test, y_train, y_test, default_score = response_common_data(
        load_iris(), SVC()
    )

    def objective(params):
        gamma, C = params
        estimator = SVC()
        estimator.set_params(gamma=gamma, C=C)

        return -np.mean(cross_val_score(estimator, X_train, y_train, cv=5,
                                        n_jobs=-1))
    space = [(1e-2, 1e2),
             (1e-2, 1e2)]
    res_gp = gp_minimize(objective, space, n_calls=100, random_state=0)
    return (default_score, -res_gp.fun)


def boston():
    X_train, X_test, y_train, y_test, default_score = response_common_data(
        load_boston(),
        SVR()
    )

    def objective(params):
        gamma, C = params
        estimator = SVR()
        estimator.set_params(gamma=gamma, C=C)

        return -np.mean(cross_val_score(estimator, X_train, y_train, cv=5,
                                        n_jobs=-1, scoring='r2'))
    space = [(1e-2, 1e2),
             (1e-2, 1e2)]
    res_gp = gp_minimize(objective, space, n_calls=100, random_state=0)
    return (default_score, -res_gp.fun)


def diabetes():
    X_train, X_test, y_train, y_test, default_score = response_common_data(
        load_diabetes(),
        SVR()
    )

    def objective(params):
        gamma, C = params
        estimator = SVR()
        estimator.set_params(gamma=gamma, C=C)

        return -np.mean(cross_val_score(estimator, X_train, y_train, cv=5,
                                        n_jobs=-1, scoring='r2'))
    space = [(1e-2, 1e2),
             (1e-2, 1e2)]
    res_gp = gp_minimize(objective, space, n_calls=100, random_state=0)
    return (default_score, -res_gp.fun)


def digits():
    X_train, X_test, y_train, y_test, default_score = response_common_data(
        load_digits(), SVC()
    )

    def objective(params):
        gamma, C = params
        estimator = SVC()
        estimator.set_params(gamma=gamma, C=C)

        return -np.mean(cross_val_score(estimator, X_train, y_train, cv=5,
                                        n_jobs=-1))
    space = [(1e-2, 1e2),
             (1e-2, 1e2)]
    res_gp = gp_minimize(objective, space, n_calls=100, random_state=0)
    return (default_score, -res_gp.fun)


def response_common_data(dataset, estimator):
    X_train, X_test, y_train, y_test = train_test_split(dataset.data,
                                                        dataset.target,
                                                        test_size=TEST_SIZE,
                                                        random_state=0)
    default_score = calculate_score(estimator, X_train, X_test, y_train,
                                    y_test)
    return (X_train, X_test, y_train, y_test, default_score)

In [3]:
# 左がデフォルトパラメータ利用時の正解率あるいはR2、右がベイズ最適化利用時の正解率あるいはR2
iris_result = iris()
iris_result

(0.97777777777777775, 0.9799498746867169)

In [4]:
boston_result = boston()
boston_result

(0.01018093344367077, 0.3620175771317663)

In [5]:
diabetes_result = diabetes()
diabetes_result

(-0.025718407007948363, 0.53852487160704732)

In [6]:
digits_result = digits()
digits_result

(0.47222222222222221, 0.83363376134337219)